In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [2]:
offers_connection = sqlite3.connect('../datasets/offers_dataset.db')
offers_frame = pd.read_sql_query('''SELECT * FROM offers''', offers_connection)
offers_frame.set_index('ID', inplace=True)
offers_connection.close()

# [!] Le descrizioni all'interno del notebook non sono mostrate nella loro versione completa
#     per accedere ad una specifica descrizione usare il metodo: offers_frame.loc[ID, NomeColonna]
offers_frame

,Name,Description,Location
ID,,,
1,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria"
2,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona
3,Software Developer,We are looking for talented and passionate peo...,55100 Lucca
4,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata"
5,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia"
...,...,...,...
850,JUNIOR DEVELOPER,Sede lavoro: Bergamo | Tempo pieno \n Livello:...,"Bergamo, Lombardia"
851,Sviluppatore Java Junior,TRIA ricerca uno sviluppatore Java JuniorNeola...,None
852,Stage Software Developer,CHI SIAMO \n Golilla è la start up delle azi...,20089 Rozzano


## Data Exploration
Come prima cosa esaminiamo il nostro dataset di offerte, in particolare dovremmo verificare la qualità dei dati:
- Base   
    - Presenza di valori nulli
    - (**Importante**) Presenza di sinonimi geografici
    - Presenza di informazioni utili
- Avanzato
    - Analisi delle Parole Chiave (es. Titoli)
    - Analisi della distribuzione dei dati

**Presenza di Valori Nulli**
Come è possibile vedere di seguito il dataset contiene valori nulli solo per la colonna Location, il che però è normale, in quanto non tutte le Offerte di lavoro sono in presenza, quindi le Location mancanti rappresentano il numero di Offerte di lavoro in Remote.
Questa informazione è importante perchè in seguito ci sarà la necessità di rappresentare queste informazioni in modo omogeneo, in modo tale che un modello il cui scopo è raggruppare le Offerte (clustering, come proposto nell'Analisi del Problema) possa farlo in modo adeguato.

In [3]:
offers_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 854 entries, 1 to 854
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         854 non-null    object
 1   Description  854 non-null    object
 2   Location     700 non-null    object
dtypes: object(3)
memory usage: 26.7+ KB


## Presenza di Sinonimi Geografici
Il dataset contiene attualmente molteplici istanze in cui la stessa Location è riportata in modi diversi, come \<Cap>, \<Città> e \<Città>, \<Regione>.


In [4]:
#example_city = "Bologna"

#onsite_offers = offers_frame[offers_frame["Location"].notna()]
#geo_syn = onsite_offers.Location.map(lambda text: example_city in text)
#onsite_offers[geo_syn]

**Analisi dei Titoli**
Di seguito vengono analizzati i Titoli delle Offerte di lavoro con lo scopo di verificare la presenza di termini comuni, in particolare vengono usate delle basilari tecniche di Natural Language Processing come:
- **Tokenization**, utilizzata per dividere frasi o documenti in parole (*title.split()*) 
- **Rimozione della Punteggiatura**, utilizzata per migliorare la qualità dei dati estratti, in modo da non influenzare i risultati 
- **Normalizzazione**, utilizzata per ridurre la complessità del vocabolario e trattare parole con maiuscole e minuscole come equivalenti (*t.lower()*)

In [5]:
from string import punctuation
from collections import Counter

def count_words(document: str, counter: Counter):
    """
    :param document: a string representation of the document to inspect
    :param counter: a Python built-in object used to count occurrences of items
    """
    punct = [p for p in punctuation]
    for t in document.split():
        if not t in punct:
            counter[t.lower()] += 1


titles_counts = Counter()
offers_frame.Name.map(lambda text: count_words(text, titles_counts))

titles_counts

Counter({'software': 582,
         'developer': 522,
         'junior': 122,
         'engineer': 106,
         'senior': 83,
         'c#': 81,
         'java': 77,
         'sviluppatore': 73,
         'programmatore': 60,
         'settore': 54,
         'sanità': 54,
         'frontend': 30,
         'full': 28,
         'ai': 27,
         'trainer': 27,
         '(contract)': 27,
         'tecnico': 27,
         'bilance': 27,
         'digitali': 27,
         'supermercati': 27,
         'asp.net': 27,
         'manufacturing': 27,
         'relocation': 27,
         'usa': 27,
         'front-end': 27,
         'app': 27,
         'programmer': 27,
         'hmi': 27,
         'stage': 23,
         'web': 23,
         'graduate': 23,
         'ibp': 22,
         'algorithms': 22,
         'development': 22,
         'unity': 22,
         'game': 22,
         'remote': 19,
         'appartenente': 18,
         'alle': 18,
         'categorie': 18,
         'protette': 18,
       

Analisi delle occorrenze di parole:

In [6]:
# Jacopo plotta qui

### Skill Extraction
Per procedere con l'estrazione delle Competenze dalle Offerte per prima cosa importiamo il relativo dataset.   

In [7]:
with sqlite3.connect('../datasets/skills_dataset.db') as skills_connection:
    skills_frame = pd.read_sql_query('SELECT * FROM skills', skills_connection)
    skills_frame.set_index('ID', inplace=True)
    
skills_frame

,SKILL,TYPE
ID,,
0,JavaScript,Programming Language
1,HTML,Programming Language
2,Python,Programming Language
3,SQL,Programming Language
4,TypeScript,Programming Language
...,...,...
95,APT,Tool
96,Unity 3D,Tool
97,Pacman,Tool


In [8]:
test_descriptions = offers_frame.loc[:5, 'Description']
skills_list = [skill.lower() for skill in skills_frame['SKILL'].tolist()]

removal = {p: '' for p in punctuation}
removal['\n'] = ''

# Remove punctuation and \n
t = test_descriptions.iloc[1]
for old, new in removal.items():
    t = t.replace(old, new)

t

'NETtoWORK azienda italiana nata nel 2016 operante nel settore dei servizi si occupa di soddisfare le necessità dei clienti offrendo servizi personalizzati   Il nostro team si avvale anche di consulenti esperti nel settore per supportare il cliente durante il periodo di collaborazionePosizionePer azienda cliente ricerchiamo una figura da inserire come  Junior Software DeveloperLa risorsa inserita in un team giovane e dinamico si occuperà dello sviluppo e del test di applicativi software per sistemi di comando controllo e diagnostica in ambito ferroviario ed automotive   Le attività di sviluppo e test del software saranno accompagnate anche dalla redazione dei documenti necessari per descrivere le funzioni sviluppate nel software stesso e la relativa aderenza ai requisiti di origineRequisitiE richiesta una laurea in  ingegneria triennale o magistrale anche senza esperienza lavorativa  oppure un  diploma tecnico superiore con esperienze lavorative pregresse in ambito ICTSono inoltre rich